In [1]:
import pandas as pd
import bqplot
import traitlets
import ipywidgets

In [2]:
marvel = pd.read_csv("../data/marvel-wikia-data.csv", parse_dates = ['Year'])

In [4]:
dc = pd.read_csv("../data/dc-wikia-data.csv", parse_dates = ['YEAR'])

In [5]:
marvel.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'Year'],
      dtype='object')

In [6]:
dc.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST APPEARANCE', 'YEAR'],
      dtype='object')

In [7]:
dc.rename(columns = {'FIRST APPEARANCE': 'FIRST_APPEARANCE'}, inplace=True)

In [8]:
marvel.rename(columns = {'FIRST APPEARANCE': 'FIRST_APPEARANCE', 'Year': 'YEAR'}, inplace=True)

In [9]:
marvel.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST_APPEARANCE', 'YEAR'],
      dtype='object')

In [10]:
dc.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'FIRST_APPEARANCE', 'YEAR'],
      dtype='object')

In [11]:
dc.dtypes, marvel.dtypes

(page_id                      int64
 name                        object
 urlslug                     object
 ID                          object
 ALIGN                       object
 EYE                         object
 HAIR                        object
 SEX                         object
 GSM                         object
 ALIVE                       object
 APPEARANCES                float64
 FIRST_APPEARANCE            object
 YEAR                datetime64[ns]
 dtype: object, page_id                      int64
 name                        object
 urlslug                     object
 ID                          object
 ALIGN                       object
 EYE                         object
 HAIR                        object
 SEX                         object
 GSM                         object
 ALIVE                       object
 APPEARANCES                float64
 FIRST_APPEARANCE            object
 YEAR                datetime64[ns]
 dtype: object)

In [12]:
marvel.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST_APPEARANCE,YEAR
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,Aug-62,1962-01-01
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,Mar-41,1941-01-01
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,Oct-74,1974-01-01
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,Mar-63,1963-01-01
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,Nov-50,1950-01-01


In [13]:
marvel["HAIR"].unique()

array(['Brown Hair', 'White Hair', 'Black Hair', 'Blond Hair', 'No Hair',
       'Blue Hair', 'Red Hair', 'Bald', 'Auburn Hair', 'Grey Hair',
       'Silver Hair', 'Purple Hair', 'Strawberry Blond Hair',
       'Green Hair', 'Reddish Blond Hair', 'Gold Hair', nan,
       'Orange Hair', 'Pink Hair', 'Variable Hair', 'Yellow Hair',
       'Light Brown Hair', 'Magenta Hair', 'Bronze Hair', 'Dyed Hair',
       'Orange-brown Hair'], dtype=object)

In [14]:
marvel["APPEARANCES"].describe()

count    15280.000000
mean        17.033377
std         96.372959
min          1.000000
25%          1.000000
50%          3.000000
75%          8.000000
max       4043.000000
Name: APPEARANCES, dtype: float64

In [15]:
marvel[marvel["APPEARANCES"] < 10]["name"]

3342                    Leonard Gade (Earth-616)
3343                          Merzah (Earth-616)
3344                 William Carmody (Earth-616)
3345                        Dan Kane (Earth-616)
3346               George Washington (Earth-616)
                          ...                   
15275                    Visher-Rakk (Earth-616)
15276    Weapon Red (Chinese Mutant) (Earth-616)
15277             Wolfgang von Roehm (Earth-616)
15278                            Xan (Earth-616)
15279                         Zamira (Earth-616)
Name: name, Length: 11938, dtype: object

In [16]:
min_appearances = ipywidgets.IntSlider(min = 0, max = marvel["APPEARANCES"].max())

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical')

base_hist = bqplot.Hist(sample = marvel["APPEARANCES"],
                        scales = {'sample': x_sc, 'count': y_sc},
                        bins = 32,
                        colors = ['orange'], opacities = [0.5])

hist = bqplot.Hist(sample = marvel["APPEARANCES"],
                   scales = {'sample': x_sc, 'count': y_sc},
                   bins = 32)

fig = bqplot.Figure(marks = [base_hist, hist], axes = [x_ax, y_ax])

def on_slider_change(event):
    hist.sample = marvel[marvel["APPEARANCES"] > event['new']]["APPEARANCES"]

min_appearances.observe(on_slider_change, ['value'])

display(ipywidgets.VBox([min_appearances, fig]))

In [25]:
first_appearance_sc = bqplot.DateScale()
appearance_count_sc = bqplot.LogScale()

first_appearance_ax = bqplot.Axis(scale = first_appearance_sc)
appearance_count_ax = bqplot.Axis(scale = appearance_count_sc, orientation='vertical')

new_marvel = marvel.groupby("YEAR").sum()
lines = bqplot.Lines( x = new_marvel.index, y = new_marvel["APPEARANCES"],
                         scales = {'x': first_appearance_sc, 'y': appearance_count_sc})

year_selector = bqplot.interacts.FastIntervalSelector(scale = first_appearance_sc)

histogram_count_sc = bqplot.LinearScale()
character_count_sc = bqplot.LinearScale()

hist = bqplot.Hist(sample = marvel["APPEARANCES"],
                   scales = {'sample': character_count_sc,
                             'count': histogram_count_sc})
hist_ax_x = bqplot.Axis(scale = character_count_sc, label = '# appearances', tick_rotate = 45)
hist_ax_y = bqplot.Axis(scale = histogram_count_sc, orientation='vertical', label = '# characters')

fig1 = bqplot.Figure(marks = [lines], axes = [first_appearance_ax, appearance_count_ax],
                   interaction = year_selector)
fig2 = bqplot.Figure(marks = [hist], axes = [hist_ax_x, hist_ax_y])

def interval_selection_changed(event):
    if len(event['new']) == 0: return
    min_year = event['new'][0]
    max_year = event['new'][1]
    good_selection = (marvel["YEAR"] >= min_year) & (marvel["YEAR"] <= max_year)
    hist.sample = marvel[good_selection]["APPEARANCES"]

year_selector.observe(interval_selection_changed, ['selected'])

display(ipywidgets.HBox([fig1, fig2]))

In [26]:
import numpy as np

In [35]:
arr = np.random.random((8, 8))

color_sc = bqplot.ColorScale(colors = ["black", "white"])

heatmap = bqplot.GridHeatMap(color = arr, scales = {'color': color_sc},
                            row = ["one", "two", "three", "four", "five", "six", "seven", "eight"],
                            column = ["a", "b", "c", "d", "e", "f", "g", "h"])
fig = bqplot.Figure(marks = [heatmap])
display(fig)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, marks=[GridHeatMap(color=array([[0.27884…